In [1]:
import sys
from PIL import Image, ImageTk
from tkinter import Tk, Label, LabelFrame, Button, filedialog
import cv2
import numpy as np
from random import randint
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.color import rgb2gray, rgb2hsv
from skimage.morphology import area_opening
from skimage.exposure import histogram
from skimage.filters import threshold_otsu
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [57]:
window = Tk()

window.geometry("1000x425")
window.configure(bg = "#21302d")

frame = LabelFrame(window, text = 'Image View')
frame.place(
    x = 3, y = 0,
    width = 465,
    height= 421
)
image_original = ""
label_original = ""
def loadimage():
    global image_original, label_original
    filename = filedialog.askopenfilename(initialdir="C:/Users/Adi/Documents/skripsi")
    if filename != "":
        img = Image.open(filename)
        image_original = img
        img = img.resize((853, 550))
        load = ImageTk.PhotoImage(img)
        
        label1 = Label(image = load)
        label1.image = load
        label1.place(
            x = 7, y =20,
            width= 455,
            height= 410
        )
        label_original = label1

def save_sapi():
    #print(image_original)
    value = randint(0, 1000)
    convert = cv2.cvtColor(np.array(image_original), cv2.COLOR_RGB2BGR)
    save_sapi = cv2.imwrite("./Dataset/Sapi/"+str(value)+".jpg", convert)
    label_original.configure(image="")

def save_babi():
    #print(image_original)
    value = randint(0, 1000)
    convert = cv2.cvtColor(np.array(image_original), cv2.COLOR_RGB2BGR)
    save_babi = cv2.imwrite("./Dataset/Babi/"+str(value)+".jpg", convert)
    label_original.configure(image="")

def masked_image(image, mask):
    r = image[:,:,0] * mask
    g = image[:,:,1] * mask
    b = image[:,:,2] * mask
    return np.dstack([r,g,b])
    
def training():
    path = 'Dataset'
    save_path = 'Dataset_Thresholding/'
    for filename in os.listdir(path):
        if not os.path.isdir(save_path + filename):
            os.mkdir(save_path + filename)
        for index, imagename in enumerate(os.listdir(os.path.join(path, filename))):
            image = imread(os.path.join(path, filename, imagename))
            gray = rgb2gray(image)
            thresh = threshold_otsu(gray)
            masking  = gray < thresh
            filtered = masked_image(image, masking)
            image = Image.fromarray(filtered)
            image = image.resize((778, 437))
            image = np.asarray(image)

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            cv2.imwrite(f'{save_path}/{filename}/{filename}_{index + 1}.jpg', image)
    
def hsv_process():
    path_threshold = 'Dataset_Thresholding'
    save_hsv = 'HSV_Value/'
    for filename in os.listdir(path_threshold):
        if not os.path.isdir(save_hsv + filename):
            os.mkdir(save_hsv + filename)
        for imagename in os.listdir(os.path.join(path_threshold, filename)):
            image = imread(os.path.join(path_threshold, filename, imagename))

            image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            
            cv2.imwrite(f'{save_hsv}/{filename}/{imagename}', image)
            
    path_hsv = 'HSV_Value/'
    data_hsv = {}
    for filename in os.listdir(path_hsv):
        for imagename in os.listdir(os.path.join(path_hsv, filename)):
            image = cv2.imread(os.path.join(path_hsv, filename, imagename))

            h,s,v = cv2.split(image)
            hmean = h.mean()
            smean = s.mean()
            vmean = v.mean()
            data_hsv[f'{imagename}'] = (hmean, smean, vmean, filename)
            
    file = open('Enkoding_KNN/hsv_value.pkl', 'wb')
    pickle.dump(data_hsv, file)
    file.close()
    
btn_open = Button(text = 'BROWSE', relief = 'flat',
        background = "#ffffff",
        fg = "#000000",
        command = loadimage,
        font=("Hervetica", 12)              
                 
    )
btn_open.place(
    x = 740, y = 5,
    width = 255,
    height = 27

    )

btn_process = Button(text = 'PROCESS', relief = 'flat',
        background = "#ffffff",
        fg = "#000000",
        #command = process,
        font=("Hervetica", 12)              
                 
    )
btn_process.place(
    x = 740, y = 95,
    width = 254,
    height = 27

    )

btn_save_babi = Button(text = 'SAVE BABI', relief = 'flat',
        background = "#ffffff",
        fg = "#000000",
        command = save_babi,
        font=("Hervetica", 12)              
                 
    )
btn_save_babi.place(
    x = 869, y = 35,
    width = 125,
    height = 27

    )
btn_save_sapi = Button(text = 'SAVE SAPI', relief = 'flat',
        background = "#ffffff",
        fg = "#000000",
        command = save_sapi,
        font=("Hervetica", 12)              
                 
    )
btn_save_sapi.place(
    x = 740, y = 35,
    width = 125,
    height = 27

    )

btn_thresholding = Button(text = 'THRESHOLDING', relief = 'flat',
        background = "#ffffff",
        fg = "#000000",
        command = training,
        font=("Hervetica", 12)              
                 
    )
btn_thresholding.place(
    x = 740, y = 65,
    width = 125,
    height = 27

    )

btn_hsv = Button(text = 'HSV', relief = 'flat',
        background = "#ffffff",
        fg = "#000000",
        command = hsv_process,
        font=("Hervetica", 12)              
                 
    )
btn_hsv.place(
    x = 869, y = 65,
    width = 125,
    height = 27

    )

frame_thresholding = LabelFrame(window, text = 'Image Thresholding')
frame_thresholding.place(
    x = 472, y = 0,
    width = 263,
    height= 208
)

frame_hsv = LabelFrame(window, text = 'Image HSV')
frame_hsv.place(
    x = 472, y = 213,
    width = 263,
    height= 208
)

frame_value = LabelFrame(window, text = 'Value')
frame_value.place(
    x = 740, y = 130,
    width = 253,
    height= 235
)

frame_result = LabelFrame(window, text = 'Result')
frame_result.place(
    x = 740, y = 371,
    width = 253,
    height= 50
)

window.resizable(False, False)
window.mainloop()